*Build a Agent*

Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In [1]:
# Import relevant functionality
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [5]:
# Use the agent
config = {"configurable": {"thread_id": "abc1234"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Rohit! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im Rohit! and i live in sf
================================== Ai Message ==================================

Hi Rohit! Nice to meet you. Is there anything I can help you with today?


In [6]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (662ecd68-6674-44ea-951d-49c3a8a19397)
 Call ID: 662ecd68-6674-44ea-951d-49c3a8a19397
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Monday, April 14, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-14", "content": "San Francisco, California Weather: Monday, April 14, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 54°. Precipitation 0 %.", "score": 0.9657998}, {"title": "Weather in San Francisco in April 2025 (California)", "url": "https://world-weather.info/forecast/usa/san_francisco/april-2025/", "content": "Weather in San Francisco in 

In [9]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
# os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [10]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

*Define tools*

We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1744634292, 'localtime': '2025-04-14 05:38'}, 'current': {'last_updated_epoch': 1744633800, 'last_updated': '2025-04-14 05:30', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 151, 'wind_dir': 'SSE', 'pressure_mb': 1013.0, 'pressure_in': 29.91, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 89, 'cloud': 25, 'feelslike_c': 11.5, 'feelslike_f': 52.6, 'windchill_c': 9.3, 'windchill_f': 48.8, 'heatindex_c': 9.3, 'heatindex_f': 48.7, 'dewpoint_c': 9.0, 'dewpoint_f': 48.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 3.5, 'gust_kph': 5.6}}", 'score': 0

*Using Language Models*

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [12]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-1.5-pro-latest", model_provider="google_genai")

You can call the language model by passing in a list of messages. By default, the response is a content string.

In [13]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi! How can I help you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [14]:
model_with_tools = model.bind_tools(tools)

In [15]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I help you today?
ToolCalls: []


In [16]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': 'c7d423e9-9beb-4b69-b9e3-53be8f53b84d', 'type': 'tool_call'}]


*Create the agent*

Now that we have defined the tools and the LLM, We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

In [17]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

*Run the agent*

let's see how it responds when there's no need to call a tool:

In [18]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='006c19dd-5f0a-47c2-936a-a3699d95c43c'),
 AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-9f6147f1-9dfb-4816-a6d6-8f1f69930a95-0', usage_metadata={'input_tokens': 49, 'output_tokens': 10, 'total_tokens': 59, 'input_token_details': {'cache_read': 0}})]

Let's now try it out on an example where it should be invoking the tool

In [19]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='792d63ce-5f95-46e3-a864-75ac329d7095'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-82cd4ef8-4e5d-4800-bfa5-f40f3cfa939a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': '99f12064-2cbe-4832-97c3-13e975a9f8f3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 13, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "Monday, April 14, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-14", "content": "San Francis

*Streaming Messages*

We've seen how the agent can be called with .invoke to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [20]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (ff59fa3c-9565-4f6a-a412-31729b7ff54d)
 Call ID: ff59fa3c-9565-4f6a-a412-31729b7ff54d
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Monday, April 14, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-14", "content": "San Francisco, California Weather: Monday, April 14, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 54°. Precipitation 0 %.", "score": 0.9623101}, {"title": "Weather in San Francisco in April 2025 (California)", "url": "https://world-weather.info/forecast/usa/san_francisco/april-2025/", "content": "Weather in San Francisco in April 2

*Streaming tokens*

In addition to streaming back messages, it is also useful to stream back tokens. We can do this by specifying stream_mode="messages".

::: note

Below we use message.text(), which requires langchain-core>=0.3.37.

:::

In [21]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| weather in San| Francisco is currently cloudy and overcast. The temperature is 66°F during the day and| 54°F at night. There is a 0% chance of| precipitation.|

*Adding in memory*

This agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [22]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc12345"}}

In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Bharat!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bharat! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-e276bf50-bb03-4519-ba11-a08e1436a001-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]}}
----


In [25]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You said your name is Bharat.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-42a87826-58b8-4b56-b12f-467ae900e5f9-0', usage_metadata={'input_tokens': 65, 'output_tokens': 8, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----


If you want to start a new conversation, all you have to do is change the thread_id used

In [26]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don’t have access to that information. I'm a large language model, able to communicate in response to a wide range of prompts and questions, but my knowledge about you is limited. Is there anything else I can do to help?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-2849dbf3-7d9b-4301-a84f-ec6405edac80-0', usage_metadata={'input_tokens': 51, 'output_tokens': 51, 'total_tokens': 102, 'input_token_details': {'cache_read': 0}})]}}
----
